In [ ]:
%%HTML
<style>
.jp-Notebook {
    WIDTH: 80%;
    display: flex;
    flex-direction: column;
    justify-content: center;
    margin: auto;
    font-size: 2rem;
}
iframe{
    width: 100%;
}
</style>

In [ ]:
from config_jupyter import set_wd
set_wd()

In [ ]:
import pandas as pd
import modules.scrap as scrap
import os
from glob import glob #para eliminar archivos dentro de carpeta
import plotly.graph_objects as go
import numpy as np
import itertools
from plotly.subplots import make_subplots
from typing import Literal

In [ ]:
def create_repeated_years(df:pd.DataFrame):
    start_year = 1986
    end_year = start_year + int(np.ceil(149 / 4)) - 1
    years = range(start_year, end_year + 1)
    years_repeated = np.repeat(list(years), 4)[:149]
    df['año'] = years_repeated
    return df

def create_repeated_trim(df:pd.DataFrame):
    values = ["I", "II", "III", "IV"]

    # Utilizar la función cycle para repetir los valores indefinidamente
    values_repeated = itertools.cycle(values)

    # Crear una nueva columna en el dataframe con los valores repetidos
    df["trimestre"] = [next(values_repeated) for _ in range(len(df))]
    df["período"] = df["trimestre"].astype(str) + "-" + df["año"].astype(str)
    return df

def wrangling_usos_trim():
    header = ["período","ng_v","ng_p","ng_q","bk_v","bk_p","bk_q","bi_v","bi_p","bi_q","cyl_v","cyl_p","cyl_q","abk_v","abk_p","abk_q","bc_v","bc_p","bc_q","vya_v","vya_p","vya_q"]
    df = pd.read_excel("./downloads/sh_indicesimpue_04.xls", header=4,skipfooter=6)
    df = df.dropna(axis=1)
    df.columns = header
    df = create_repeated_years(df)
    df = create_repeated_trim(df)
    df = df.set_index("período")
    df = df.drop(["año","trimestre"],axis=1)
    return df

def wrangling_usos_trim_var():
    df = wrangling_usos_trim()
    df = df.pct_change(4)
    return df.dropna()

   

In [ ]:
scrap.scrap_link_xls("https://www.indec.gob.ar/ftp/cuadros/economia/sh_indicesimpue_04.xls")

In [ ]:
def subplot_var_indice(anio_desde:str="2015"):
    anio_desde = str(anio_desde)
    # Crear el subplot con 2 columnas y 4 filas
    df = wrangling_usos_trim_var()[f"I-{anio_desde}":]
    q_columns = wrangling_usos_trim_var().columns[wrangling_usos_trim_var().columns.str.endswith("q")].drop("ng_q")
    fig = make_subplots(rows=3, cols=2, subplot_titles=["Bienes de capital","Bienes intermedios", "combustibles y lubricantes", "Acc. para bienes de capital", "Bienes de consumo", "Vehículos y automotores"])

    # Crear un bucle para añadir cada subplot
    for i, col in enumerate(q_columns, start=1):
        row = (i-1) // 2 + 1
        column = (i-1) % 2 + 1
        
        # Añadir el trace correspondiente al subplot
        trace = go.Scatter(x=df.index, y=df[col], name=col)
        fig.add_trace(trace, row=row, col=column)

    # Actualizar las propiedades del gráfico
    fig.update_layout(height=900, width=800, title='Evolución de las cantidades importadas según usos económicos<br>Variación interanual', template = None,
                    font_family = "verdana", showlegend = False)
    fig.update_yaxes(tickformat = ".0%")
    note = 'Fuente: INDEC'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.1,yref='paper', y=-0.11, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    fig.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)
    # Mostrar el gráfico
    return fig

subplot_var_indice(anio_desde=2012).show(renderer = "notebook")


In [ ]:
def subplot_indice(anio_desde:str = 2015):
    anio_desde = str(anio_desde)
# Crear el subplot con 2 columnas y 4 filas
    df = wrangling_usos_trim()[f"I-{anio_desde}":]
    q_columns = wrangling_usos_trim().columns[wrangling_usos_trim().columns.str.endswith("q")].drop("ng_q")
    fig = make_subplots(rows=3, cols=2, subplot_titles=["Bienes de capital","Bienes intermedios", "combustibles y lubricantes", "Acc. para bienes de capital", "Bienes de consumo", "Vehículos y automotores"])

    # Crear un bucle para añadir cada subplot
    for i, col in enumerate(q_columns, start=1):
        row = (i-1) // 2 + 1
        column = (i-1) % 2 + 1
        
        # Añadir el trace correspondiente al subplot
        trace = go.Scatter(x=df.index, y=df[col], name=col)
        fig.add_trace(trace, row=row, col=column)
        last_value = df[col].mean()
        fig.add_shape(
            type="line",
            x0=df.index[0],
            x1=df.index[-1],
            y0=last_value,
            y1=last_value,
            line=dict(color=trace.line.color, dash="dash"),
            row=row,
            col=column,
        )

    # Actualizar las propiedades del gráfico
    fig.update_layout(height=900, width=800, title='Evolución de las cantidades importadas según usos económicos<br>índice 2004 = 100', template = None,
                    font_family = "verdana", showlegend = False)
    # fig.update_yaxes(tickformat = ".0%")
    note = f'Fuente: INDEC. La línea punteada representa la media de {anio_desde} en adelante'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.7,yref='paper', y=-0.11, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    fig.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)

    return fig

subplot_indice(anio_desde=2012).show(renderer = "notebook")

In [ ]:
# from collections import Counter
# df = wrangling_usos_trim()
# df['trimestre'] = df.index.str.split('-').str[0]
# df["anio"] = df.index.str.split('-').str[1]
# df = df.reset_index()
# fin_de_subseries = [
#         list(df[df.trimestre==trimestre].index)[0] +
#         (list(df[df.trimestre==trimestre].index)[-1] - list(df[df.trimestre==trimestre].index)[0])/2
#         for trimestre in Counter(df.trimestre).keys()
#         ]
# TRIMESTRES = df.trimestre.unique()
# trimestres_dic = {(i+1):trimestre for i, trimestre in enumerate(TRIMESTRES)}
# # print(Counter(df.trimestre).values())
# # print(df)
# # trimestres_output =[trimestres_dic[trimestre] for trimestre in Counter(df.trimestre).keys()]
# # fig = go.Figure()
# df

In [ ]:
DIC_INDICES= {
    'ng_v':"Valor nivel general", 'ng_p': "Precios nivel general", 'ng_q': "Cantidades nivel general",
    'bk_v': "Valor bienes de capital", 'bk_p': "Precios bienes de capital", 'bk_q':"Cantidades bienes de capital",
    'bi_v': "Valor bienes intermedio", 'bi_p': "Precios bienes intermedios", 'bi_q':"Cantidades bienes intermedios",
       'cyl_v': "Valor combustibles y lubricantes", 'cyl_p':"Precios combustibles y lubricantes", 'cyl_q':"Cantidades combustibles y lubricantes", 
       'abk_v':"Valor accesorios para bienes de capital", 'abk_p':"Precios accesorios para bienes de capital", 'abk_q':"Cantidades accesorios para bienes de capital",
       'bc_v':"Valor bienes de consumo", 'bc_p':"Precios bienes de consumo", 'bc_q':"Cantidades bienes de consumo",
       'vya_v': "Valor vehículos y automotores", 'vya_p':"Precios vehículos y automotores", 'vya_q':"Cantidades vehículos y automotores"}

In [ ]:
def violin_indice(anio_desde:str="1986",
                  indice:Literal['ng_v', 'ng_p', 'ng_q', 'bk_v', 'bk_p', 'bk_q', 'bi_v', 'bi_p', 'bi_q',
       'cyl_v', 'cyl_p', 'cyl_q', 'abk_v', 'abk_p', 'abk_q', 'bc_v', 'bc_p',
       'bc_q', 'vya_v', 'vya_p', 'vya_q'] = "ng_v"):
    df = wrangling_usos_trim()
    df['trimestre'] = df.index.str.split('-').str[0]
    df["anio"] = df.index.str.split('-').str[1]
    df = df[df.anio >= str(anio_desde)]
    fig = go.Figure()
    for trimestre in df.trimestre.unique():
        fig.add_trace(go.Violin(x=df['trimestre'][df['trimestre'] == trimestre],
                            y=df['ng_v'][df['trimestre'] == trimestre],
                            name=trimestre,
                            customdata=df['anio'][df['trimestre'] == trimestre],
                            hovertemplate='Anio: %{customdata}<br>Value: %{y}',
                            box_visible=True,
                            line_color='black',
                            fillcolor='lightseagreen',
                            points='all', 
                            meanline_visible=True))
    fig.update_layout(template = None, title_text = f"Violin plot. Indice {DIC_INDICES[indice].lower()}, desde {anio_desde}",
                      showlegend = False)
    return fig
violin_indice(anio_desde=2015, indice="ng_q").show(renderer = "notebook")

In [ ]:
def plot_seasonal_analysis(anio_desde:str = '2012',
                           indice:Literal['ng_v', 'ng_p', 'ng_q', 'bk_v', 'bk_p', 'bk_q', 'bi_v', 'bi_p', 'bi_q',
       'cyl_v', 'cyl_p', 'cyl_q', 'abk_v', 'abk_p', 'abk_q', 'bc_v', 'bc_p',
       'bc_q', 'vya_v', 'vya_p', 'vya_q'] = "ng_v"):
    df = wrangling_usos_trim()
    df['trimestre'] = df.index.str.split('-').str[0]
    df["anio"] = df.index.str.split('-').str[1]

    fig = go.Figure()
    for i,año in enumerate(df[df.anio >= str(anio_desde)].anio.unique()):
        trace = df.query("anio == @año")
        fig.add_trace(go.Scatter(x = trace.trimestre, y = trace.ng_v, name=año, marker_symbol=i))
    fig.update_layout(template = None, title_text = f"Seasonal analysis. Índice: {indice}, desde {anio_desde}", font_family = "verdana")
    return fig

plot_seasonal_analysis(anio_desde=2012, indice="ng_v").show(renderer = "notebook")

In [ ]:
def violin_indice(anio_desde: str = "1986"):
    df = wrangling_usos_trim()
    df['trimestre'] = df.index.str.split('-').str[0]
    df["anio"] = df.index.str.split('-').str[1]
    df = df[df.anio >= str(anio_desde)]

    num_indices = len(DIC_INDICES)
    num_rows = (num_indices + 2) // 3  # Calcular el número de filas necesario

    # Crear un subplot con 3 columnas y la cantidad de filas necesarias
    fig = make_subplots(rows=num_rows, cols=3, subplot_titles=list(DIC_INDICES.values()))

    row = 1
    col = 1
    for indice in DIC_INDICES.keys():
        for trimestre in df.trimestre.unique():
            fig.add_trace(go.Violin(
                x=df['trimestre'][df['trimestre'] == trimestre],
                y=df[indice][df['trimestre'] == trimestre],
                name=trimestre,
                customdata=df['anio'][df['trimestre'] == trimestre],
                hovertemplate='Anio: %{customdata}<br>Value: %{y}',
                box_visible=True,
                line_color='black',
                fillcolor='lightseagreen',
                points='all',
                meanline_visible=True
            ), row=row, col=col)


        col += 1
        if col > 3:
            col = 1
            row += 1

    fig.update_xaxes(title_text="Trimestre")
    fig.update_layout(title=f"Violin plot. I. Desde {anio_desde}", showlegend=False,
                      height = 1400, template = None)
    return fig

violin_indice(anio_desde=2015).show(renderer="notebook")


In [ ]:
delete = input("Escribir algo si se quiere eliminar el xls, sino apretar enter")
if delete:
    for i in glob("../downloads/*", recursive = True): os.remove(i)

In [ ]:
%%cmd
jupyter nbconvert --no-input --to html --output-dir=../output indices_trim_scrap.ipynb